## iii-org rancher pipeline data generator example

透過服務函式建立完整rancher pipeline資料內容(教學範例)

#### 參考
* [使用-python-讀取-yaml-檔案-d3f413d7dd6](https://medium.com/bryanyang0528/%E4%BD%BF%E7%94%A8-python-%E8%AE%80%E5%8F%96-yaml-%E6%AA%94%E6%A1%88-d3f413d7dd6)
* [yaml.readthedocs](https://yaml.readthedocs.io/en/latest/example.html)

## 引入函式庫

In [1]:
import yaml
import sys
import os

In [2]:
## 
PIPELINE_TEMPLATE_FILE = './stander-rancher-pipeline.yaml'
## 部屬環境Config模擬
DEVOPS_REGISTRY = os.getenv('DEVOPS_REGISTRY', "XX.XX.XX.XX:XXXX")
REGISTRY_INSECURE = os.getenv('REGISTRY_INSECURE', "false")
GIT_HOST = os.getenv('GIT_HOST', "10.XX.XX.XX")

## 讀取標準範例樣板並顯示樣板

In [3]:
with open(PIPELINE_TEMPLATE_FILE, 'r') as stream:
    pipeline_template = yaml.load(stream, Loader=yaml.FullLoader)
yaml.dump(pipeline_template, sys.stdout)

stages:
- name: Publish User Web image
  steps:
  - env:
      PLUGIN_INSECURE: 'true'
    publishImageConfig:
      buildContext: .
      dockerfilePath: ./Dockerfile
      pushRemote: true
      registry: 10.50.1.63:5443
      tag: app/${CICD_GIT_REPO_NAME}-web:latest
- name: Deploy app-web
  steps:
  - applyAppConfig:
      answers:
        checkmarx.enabled: false
        git.branch: ${CICD_GIT_BRANCH}
        git.commitID: ${CICD_GIT_COMMIT}
        git.repoName: ${CICD_GIT_REPO_NAME}
        git.url: ${CICD_GIT_URL}
        newman.enabled: false
        web.image: 10.50.1.63:5443/app/${CICD_GIT_REPO_NAME}-web
        web.tag: latest
        web.type: uwsgi-flask
      catalogTemplate: cattle-global-data:iii-dev-charts3-web
      name: ${CICD_GIT_REPO_NAME}-${CICD_GIT_BRANCH}-web
      targetNamespace: p-v2nzs-pipeline
      version: 0.1.0
- name: Deploy app-db
  steps:
  - applyAppConfig:
      answers:
        db.name: health
        db.password: bearisbear
        db.type: mysq

## 按照部屬環境進行範本環境的修改

In [14]:
def env_rancher_pipeline_generate(PIPELINE_TEMPLATE_FILE, DEVOPS_REGISTRY, REGISTRY_INSECURE, GIT_HOST):
    # 讀取範例檔案 
    with open(PIPELINE_TEMPLATE_FILE, 'r') as stream:
        pipeline_template = yaml.load(stream, Loader=yaml.FullLoader)
    temp_pipeline_stages = pipeline_template['stages']
    # 修改第一階段 Publish User Web image
    temp_pipeline_stages[0]['steps'][0]['publishImageConfig']['registry'] = DEVOPS_REGISTRY
    temp_pipeline_stages[0]['steps'][0]['env']['PLUGIN_INSECURE'] = REGISTRY_INSECURE
    # 修改第三個階段 
    temp_pipeline_stages[2]['steps'][0]['applyAppConfig']['answers']['git.host'] = GIT_HOST
    return pipeline_template

In [15]:
yaml.dump(env_rancher_pipeline_generate(PIPELINE_TEMPLATE_FILE, DEVOPS_REGISTRY, REGISTRY_INSECURE, GIT_HOST), sys.stdout)

stages:
- name: Publish User Web image
  steps:
  - env:
      PLUGIN_INSECURE: 'false'
    publishImageConfig:
      buildContext: .
      dockerfilePath: ./Dockerfile
      pushRemote: true
      registry: XX.XX.XX.XX:XXXX
      tag: app/${CICD_GIT_REPO_NAME}-web:latest
- name: Deploy app-web
  steps:
  - applyAppConfig:
      answers:
        checkmarx.enabled: false
        git.branch: ${CICD_GIT_BRANCH}
        git.commitID: ${CICD_GIT_COMMIT}
        git.repoName: ${CICD_GIT_REPO_NAME}
        git.url: ${CICD_GIT_URL}
        newman.enabled: false
        web.image: 10.50.1.63:5443/app/${CICD_GIT_REPO_NAME}-web
        web.tag: latest
        web.type: uwsgi-flask
      catalogTemplate: cattle-global-data:iii-dev-charts3-web
      name: ${CICD_GIT_REPO_NAME}-${CICD_GIT_BRANCH}-web
      targetNamespace: p-v2nzs-pipeline
      version: 0.1.0
- name: Deploy app-db
  steps:
  - applyAppConfig:
      answers:
        db.name: health
        db.password: bearisbear
        db.type: my

## 使用者僅需要前端部屬

In [23]:
def web_service_pipeline(checkmarxEnabled, newmanEnabled, webImage, webTag, webType):
    global PIPELINE_TEMPLATE_FILE, DEVOPS_REGISTRY, REGISTRY_INSECURE, GIT_HOST
    pipeline_template = env_rancher_pipeline_generate(PIPELINE_TEMPLATE_FILE, DEVOPS_REGISTRY, REGISTRY_INSECURE, GIT_HOST)
    temp_pipeline_stages = pipeline_template['stages']
    # 把最後面的階段移除就會變成結果
    temp_pipeline_stages.pop(2)
    # 將使用者參數帶入
    app_web_answers = temp_pipeline_stages[1]['steps'][0]['applyAppConfig']['answers']
    app_web_answers['checkmarx.enabled'] = checkmarxEnabled
    app_web_answers['newman.enabled'] = newmanEnabled
    app_web_answers['web.image'] = webImage
    app_web_answers['web.tag'] = webTag
    app_web_answers['web.type'] = webType
    return yaml.dump(pipeline_template, sys.stdout)

In [24]:
web_service_pipeline(True, False, 'bearimage', 'beartag', 'php')

stages:
- name: Publish User Web image
  steps:
  - env:
      PLUGIN_INSECURE: 'false'
    publishImageConfig:
      buildContext: .
      dockerfilePath: ./Dockerfile
      pushRemote: true
      registry: XX.XX.XX.XX:XXXX
      tag: app/${CICD_GIT_REPO_NAME}-web:latest
- name: Deploy app-web
  steps:
  - applyAppConfig:
      answers:
        checkmarx.enabled: true
        git.branch: ${CICD_GIT_BRANCH}
        git.commitID: ${CICD_GIT_COMMIT}
        git.repoName: ${CICD_GIT_REPO_NAME}
        git.url: ${CICD_GIT_URL}
        newman.enabled: false
        web.image: bearimage
        web.tag: beartag
        web.type: php
      catalogTemplate: cattle-global-data:iii-dev-charts3-web
      name: ${CICD_GIT_REPO_NAME}-${CICD_GIT_BRANCH}-web
      targetNamespace: p-v2nzs-pipeline
      version: 0.1.0


## 使用者僅需要資料庫

In [25]:
def db_service_pipeline(db_name, db_type, db_username, db_password):
    global PIPELINE_TEMPLATE_FILE, DEVOPS_REGISTRY, REGISTRY_INSECURE, GIT_HOST
    pipeline_template = env_rancher_pipeline_generate(PIPELINE_TEMPLATE_FILE, DEVOPS_REGISTRY, REGISTRY_INSECURE, GIT_HOST)
    temp_pipeline_stages = pipeline_template['stages']
    # 把最前面的兩個階段移除
    temp_pipeline_stages.pop(0)
    temp_pipeline_stages.pop(0)
    # 將使用者參數帶入
    app_web_answers = temp_pipeline_stages[0]['steps'][0]['applyAppConfig']['answers']
    app_web_answers['db.name'] = db_name
    app_web_answers['db.type'] = db_type
    app_web_answers['db.username'] = db_username
    app_web_answers['db.password'] = db_password
    return yaml.dump(pipeline_template, sys.stdout)

In [26]:
db_service_pipeline('bearDB', 'mysql', 'bearuser', 'bearpassword')

stages:
- name: Deploy app-db
  steps:
  - applyAppConfig:
      answers:
        db.name: bearDB
        db.password: bearpassword
        db.type: mysql
        db.username: bearuser
        git.branch: ${CICD_GIT_BRANCH}
        git.host: 10.XX.XX.XX
        git.repoName: ${CICD_GIT_REPO_NAME}
      catalogTemplate: cattle-global-data:iii-dev-charts3-db
      name: ${CICD_GIT_REPO_NAME}-${CICD_GIT_BRANCH}
      targetNamespace: p-v2nzs-pipeline
      version: 0.1.0
